<a href="https://colab.research.google.com/github/jtrinquier/SoftAlign/blob/main/SCOPE_SEARCH_SoftAlign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import jax
!  pip install git+https://github.com/deepmind/dm-haiku
import haiku as hk
import jax.numpy as jnp
from jax import vmap
import numpy as np
import time
import numpy as np
import time
import ENCODING as enco
import Score_align as score_
import utils

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-8rhjjv08
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/dm-haiku /tmp/pip-req-build-8rhjjv08
  Resolved https://github.com/deepmind/dm-haiku to commit d14f1b62798a4da47cf3243cd87b1ad7dc52a108
  Preparing metadata (setup.py) ... done
  Created wheel for dm-haiku: filename=dm_haiku-0.0.14.dev0-py3-none-any.whl size=373712 sha256=aad982ec94730ded3e58ababe3ac35293e3272c784acf6a30bbc17e188312a5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-av0xxrk4/wheels/86/29/0a/454c478e7217c383ee3d05aa43cbeff48284c23430f59d0c4f
Successfully built dm-haiku


If you want to work on SCOPE, run the first cell, if you have your custom pdb folder, run the second



In [1]:
 # @title SCOPE database Download here https://drive.google.com/file/d/1DFWcUgPukTxWGPUxaeTM1kNEVNCkRgbO/view?usp=sharing
# inputs of ProteinMPNN
import pickle
with open("dicti_inputs_SCOPE_colab",'rb') as fff:

  dicti_inputs = pickle.load(fff)

In [ ]:
 # @title Custom pdbs folder
pdb_folder_path = "pdb_folder" # @param {"type":"string"}


import Input_MPNN as inp
import os

def process_pdb_folder(pdb_folder, chain='A'):
    data = {}
    for filename in os.listdir(pdb_folder):
        if filename.endswith(".pdb"):
            pdb_path = os.path.join(pdb_folder, filename)
            coords, mask, chain_, res = inp.get_inputs_mpnn(pdb_path, chain=chain)
            data[filename] = (
                coords,
                mask,
                chain_,
                res
            )
            print(f"Processed {filename}")
    return data

dicti_inputs = process_pdb_folder(pdb_folder_path)
#print(dicti_inputs)

Processed 3f08.pdb
Processed 3ejv.pdb
Processed 3erw.pdb
Processed 3jtd.pdb
Processed 3f04.pdb
Processed 3edh.pdb
Processed 3jqw.pdb
Processed 3do6.pdb
Processed 3e9n.pdb
Processed 3jtg.pdb


In [2]:
# MODEL

params_path = "./CONT_SFT_06_T_3_1" #@param {type:"string"}

params= pickle.load(open(params_path,"rb"))

<ipython-input-2-87e59704d7e1>:5: DeprecationWarning: Pickled array contains an aval with a named_shape attribute. This is deprecated and the code path supporting such avals will be removed. Please re-pickle the array.
  params= pickle.load(open(params_path,"rb"))


# ENCODING

In [6]:
key = jax.random.PRNGKey(0)

num_layers = 3 #@param {type:"integer"}
num_neighbors = 64 #@param {type:"integer"}
encoding_dim = 64 #@param {type:"integer"}
categorical = False #@param {type:"boolean"}
nb_clusters = 20 #@param {type:"integer"}


def enco_(x1,node_features = encoding_dim,
                 edge_features = encoding_dim, hidden_dim = encoding_dim,
                 num_encoder_layers=num_layers,
                  k_neighbors=num_neighbors,categorical = categorical,nb_clusters = nb_clusters):
  if categorical:
      a = enco.ENCODING_KMEANS_SEQ(node_features,edge_features,hidden_dim,num_encoder_layers,k_neighbors,nb_clusters = nb_clusters)

  else:

    a = enco.ENCODING(node_features,edge_features,hidden_dim,num_encoder_layers,k_neighbors)
  return a(x1)

ENCO = hk.transform(enco_)
@jax.jit
def enco_fast(params,key,input_data):
  return ENCO.apply(params,key,input_data)

In [7]:
# @title Prepare data

X1s = []
mask1s = []
chain1s = []
res1s = []

id1s = []
l1 = []

for k in dicti_inputs.keys():
    pr1 = k
    _X1, _mask1, _chain1, _res1 = dicti_inputs[pr1]
    id1s.append(pr1)
    X1s.append(_X1[0])
    mask1s.append(_mask1[0])
    chain1s.append(_chain1[0])
    res1s.append(_res1[0])
    l1.append(len(_X1[0]))

max_len = max(l1)

# NOTE: If max_size is too large, consider splitting your data into smaller chunks

print(f"max_size set to: {max_len}")





max_size set to: 1419


In [8]:
# @title Create encodings
ENCOO = hk.transform(enco_)
encodings = []
# Set batch size
bs = 10
num_samples = len(X1s)

# Convert lists to numpy arrays
X1s = np.array(X1s,dtype=object)
mask1s = np.array(mask1s,dtype=object)
res1s = np.array(res1s,dtype=object)
chain1s = np.array(chain1s,dtype=object)

key = jax.random.PRNGKey(42)

# Start timer
beg = time.time()

# Loop over batches
for i in range(num_samples // bs):
    # Pad the current batch
    X1, mask1, res1, chain1, X2, mask2, res2, chain2, lens = utils.pad_(
        X1s[i * bs:(i + 1) * bs], mask1s[i * bs:(i + 1) * bs], res1s[i * bs:(i + 1) * bs], chain1s[i * bs:(i + 1) * bs],
        X1s[i * bs:(i + 1) * bs], mask1s[i * bs:(i + 1) * bs], res1s[i * bs:(i + 1) * bs], chain1s[i * bs:(i + 1) * bs],
        max_len
    )
    input_data = X2, mask2, res2, chain2
    encodings_ = enco_fast(params, key, input_data)

    # Directly extend the encodings list with the results
    encodings.extend(encodings_)

# Process the remaining samples if any
if num_samples % bs != 0:
    X1, mask1, res1, chain1, X2, mask2, res2, chain2, lens = utils.pad_(
        X1s[num_samples - num_samples % bs:], mask1s[num_samples - num_samples % bs:], res1s[num_samples - num_samples % bs:],
        chain1s[num_samples - num_samples % bs:], X1s[num_samples - num_samples % bs:], mask1s[num_samples - num_samples % bs:],
        res1s[num_samples - num_samples % bs:], chain1s[num_samples - num_samples % bs:], max_len
    )
    input_data = X2, mask2, res2, chain2
    encodings_ = ENCOO.apply(params, key, input_data)
    encodings.extend(encodings_)

# End timing and print
print(time.time() - beg)



dicti_encodings = {}
for l,k in enumerate(encodings):

  dicti_encodings[id1s[l]] = k[:l1[l],:]

201.38864922523499


# Search

In [9]:
import os

def write_sorted_results(input_file_path, output_dir):
    """
    Reads a results file, groups by the first column, sorts by the last column,
    and writes individual sorted files to the output directory.

    Args:
        input_file_path (str): Path to the input results file.
        output_dir (str): Directory to save the sorted files.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Read lines from the input file
    with open(input_file_path, 'r') as file:
        lines = [line.strip() for line in file]

    # Group lines by the first column
    lines_by_value = {}
    for line in lines:
        values = line.split(',')
        key = values[0]
        if key not in lines_by_value:
            lines_by_value[key] = []
        lines_by_value[key].append(line)

    # Process and write each group to separate files
    for key, lines_for_value in lines_by_value.items():
        sorted_lines = sorted(lines_for_value, key=lambda line: -float(line.split(',')[-1]))
        output_file_path = os.path.join(output_dir, f'{key}.txt')
        with open(output_file_path, 'w') as output_file:
            output_file.writelines(line + '\n' for line in sorted_lines)
        #print(f'Sorted lines for {key} have been written to {output_file_path}')

    print(f'All {len(lines_by_value)} files generated successfully.')



In [10]:
! mkdir ./search

In [11]:
import numpy as np
import jax.numpy as jnp
import time
import os

def filter_lists(X1, lengths, encodings, thresh_query, thresh_target, ids):
    min_q, max_q = thresh_query
    min_t, max_t = thresh_target
    X1_q, X1_t, enc_q, enc_t, ids_q, ids_t, len_q, len_t = [], [], [], [], [], [], [], []

    for i in range(len(X1)):
        if min_q < lengths[i] <= max_q:
            X1_q.append(X1[i])
            enc_q.append(encodings[i])
            ids_q.append(ids[i])
            len_q.append([lengths[i]])
        if min_t < lengths[i] <= max_t:
            X1_t.append(X1[i])
            enc_t.append(encodings[i])
            ids_t.append(ids[i])
            len_t.append([lengths[i]])

    print(f" - Selected {len(X1_q)} queries (range: {min_q}-{max_q}) and {len(X1_t)} targets (range: {min_t}-{max_t})")
    return X1_q, X1_t, enc_q, enc_t, len_q, len_t, ids_q, ids_t


def prepare_data(thresh_query=(0, 100), thresh_target=(0, 100)):
    X1, lengths, encodings, ids = [], [], [], []
    for k in dicti_inputs.keys():
        coords = dicti_inputs[k][0][0]
        X1.append(coords)
        lengths.append(coords.shape[0])
        encodings.append(dicti_encodings[k])
        ids.append(k)
    return filter_lists(X1, lengths, encodings, thresh_query, thresh_target, ids)


def pad_data(X_q, X_t, enc_q, enc_t, max_len_q, max_len_t):
    X1_padded = np.array([np.pad(X, ((0, max_len_q - X.shape[0]), (0, 0), (0, 0)), 'constant') for X in X_q])
    X2_padded = np.array([np.pad(X, ((0, max_len_t - X.shape[0]), (0, 0), (0, 0)), 'constant') for X in X_t])
    enc1_padded = jnp.array([jnp.pad(X, ((0, max_len_q - X.shape[0]), (0, 0)), 'constant') for X in enc_q])
    enc2_padded = jnp.array([jnp.pad(X, ((0, max_len_t - X.shape[0]), (0, 0)), 'constant') for X in enc_t])
    return X1_padded, X2_padded, enc1_padded, enc2_padded


def find_divisors(n):
    return [i for i in range(1, n + 1) if n % i == 0]


def batch_generator(X1, X2, enc1, enc2, len1, len2, batch_size):
    n1, n2 = len(enc1), len(enc2)
    X2_aug = jnp.vstack((X2, X2))
    enc2_aug = jnp.vstack((enc2, enc2))
    len2_aug = len2 * 2

    for i in range(0, n1, batch_size):
        batch_X1 = X1[i:i + batch_size]
        batch_enc1 = enc1[i:i + batch_size]
        batch_len1 = len1[i:i + batch_size]

        for j in range(0, n2, 1):
            batch_X2 = X2_aug[j:j + batch_size]
            batch_enc2 = enc2_aug[j:j + batch_size]
            batch_len2 = len2_aug[j:j + batch_size]
            yield batch_X1, batch_X2, batch_enc1, batch_len1, batch_enc2, batch_len2


def generate_pairs(ids1, ids2, batch_size):
    ids2_aug = ids2 * 2
    return [(ids1[i + l], ids2_aug[j + l]) for i in range(0, len(ids1), batch_size) for j in range(0, len(ids2)) for l in range(batch_size)]



In [53]:
def find_divisors(n):
    """Returns a list of divisors of n."""
    return [i for i in range(1, n + 1) if n % i == 0]

def smart_bin_creator(lengths, min_samples_per_bin=300, max_bin_width=5000):
    """
    Dynamically creates non-overlapping bins based on sequence length distribution.
    Ensures the number of elements inside each bin has a divisor >10 and is even.
    """
    sorted_lengths = sorted(lengths)
    bins = []
    i = 0

    while i < len(sorted_lengths):
        start = sorted_lengths[i] - 1 if i == 0 else sorted_lengths[i]  # ✅ First bin starts at -1
        count = 0
        j = i

        # Expand the bin until reaching min_samples_per_bin or max_bin_width
        while j < len(sorted_lengths) and (sorted_lengths[j] - start) <= max_bin_width:
            count += 1
            if count >= min_samples_per_bin:
                break
            j += 1

        end = sorted_lengths[min(j, len(sorted_lengths) - 1)]  # Default bin end
        num_elements = j - i  # ✅ Number of elements inside the bin

        # Ensure the number of elements meets both conditions
        while True:
            divisors = find_divisors(num_elements)

            if any(d > 10 for d in divisors) and num_elements % 16== 0:  # ✅ Check divisor >10 and even
                break  # ✅ Valid bin found

            # Expand bin if needed (avoid exceeding last length)
            if j < len(sorted_lengths) - 1:
                j += 1
                end = sorted_lengths[min(j, len(sorted_lengths) - 1)]
                num_elements = j - i  # Recalculate elements
            else:
                break  # Stop expanding if at last element

        # Ensure last bin includes the maximum length
        if j >= len(sorted_lengths) - 1:
            end = sorted_lengths[-1] + 1

        bins.append((start, end))
        i = j  # ✅ Move to the next bin without overlap

    return bins

# Example usage
all_lengths = [dicti_inputs[k][0][0, :, :, :].shape[0] for k in dicti_inputs]
print(f"Found {len(all_lengths)} total sequences.")
length_bins = smart_bin_creator(all_lengths, min_samples_per_bin=300, max_bin_width=1000)
print(f"Smart bins: {length_bins}")





Found 11211 total sequences.
Smart bins: [(4, 41), (41, 55), (55, 63), (63, 70), (70, 76), (76, 82), (82, 87), (87, 92), (92, 96), (96, 101), (101, 105), (105, 110), (110, 115), (115, 120), (120, 124), (124, 129), (129, 134), (134, 140), (140, 146), (146, 153), (153, 160), (160, 167), (167, 176), (176, 184), (184, 194), (194, 206), (206, 217), (217, 230), (230, 244), (244, 258), (258, 273), (273, 292), (292, 314), (314, 343), (343, 380), (380, 452), (452, 1420)]


In [42]:
min(all_lengths)

5

In [54]:
! rm ./search/*

In [ ]:
def smart_bin_pairs(length_bins):
    """
    Pair query bins with target bins aiming for larger batch sizes.
    Returns a list of (query_range, target_range) pairs.
    """
    bin_pairs = []
    for query_bin in length_bins:
        for target_bin in length_bins:
            bin_pairs.append((query_bin, target_bin))
    return bin_pairs

def main_search_smart(length_bins, gap=0.0, open_=0.0, min_batch_size=8):
    print(f"Running SMART search | gap: {gap} | open: {open_}")

    total_start = time.time()

    bin_pairs = smart_bin_pairs(length_bins)

    last_query_bin = None  # Track last query bin

    for query_range, target_range in bin_pairs:
        if query_range != last_query_bin:
            # 🚨 New query bin detected! Reset results and write sorted results for previous bin
            if last_query_bin is not None:
                write_sorted_results("results_search", "search/")  # Sort & save results before switching bins

            print(f"\n🆕 Starting new Query Bin: {query_range}")
            if os.path.exists("results_search"):
                os.remove("results_search")  # Delete results file when changing query bin

            last_query_bin = query_range  # Update last query bin

        print(f"\n🟢 Query Bin: {query_range} x 🔵 Target Bin: {target_range}")
        start = time.time()

        X1f, X2f, enc1f, enc2f, len1f, len2f, ids1, ids2 = prepare_data(query_range, target_range)
        if not X1f or not X2f:
            print(" ⚠️  Skipping (no data found)\n")
            continue

        max_len_q = query_range[1]
        max_len_t = target_range[1]
        X1p, X2p, enc1p, enc2p = pad_data(X1f, X2f, enc1f, enc2f, max_len_q, max_len_t)

        possible_batches = find_divisors(len(X1f))
        cond = (query_range[1] < 500 and target_range[1] < 500)

        if not cond:
            batch_size = max([b for b in possible_batches if b < 10], default=1)
        else:
            batch_size = max([b for b in possible_batches if b <= min(len(X2f), 1000) and b >= min_batch_size], default=1)

        print(f"   ➡️  Total query x target pairs: {len(X1f) * len(X2f)}, Batch size: {batch_size}")
        if batch_size < min_batch_size:
            print(" ⚠️  Not optimal size.\n")
            # continue

        res_lddt = []
        batches = batch_generator(X1p, X2p, enc1p, enc2p, len1f, len2f, batch_size)

        for idx, batch in enumerate(batches):
            batch_X1, batch_X2, batch_enc1, batch_len1, batch_enc2, batch_len2 = batch
            aln = score_.align_argmax(batch_enc1, batch_enc2, jnp.column_stack((batch_len1, batch_len2)), 1e-7)
            col_mask = np.arange(aln.shape[1])[np.newaxis, :] >= np.array(batch_len1)
            mask1 = ~col_mask[:, np.newaxis, :] * ~col_mask[:, :, np.newaxis]
            col_mask2 = np.arange(aln.shape[2])[np.newaxis, :] >= np.array(batch_len2)
            mask2 = ~col_mask2[:, np.newaxis, :] * ~col_mask2[:, :, np.newaxis]

            res_lddt.append(score_.get_LDDTloss(batch_X1[:, :, 1], batch_X2[:, :, 1], aln, mask1, mask2, 10**-4, values_ANG=[0.5, 1, 2, 4]))
            if idx % 500 == 0:
                print(f"     - Processed {idx * batch_size} pairs...")

        res_lddt_array = np.concatenate(res_lddt)
        pairs = generate_pairs(ids1, ids2, batch_size)

        with open("results_search", "a") as f:  # ✅ Append instead of overwrite
            for i in range(len(pairs)):
                f.write(f"{pairs[i][0]},{pairs[i][1]},{res_lddt_array[i]}\n")

        print(f" ✅ Bin {query_range} x {target_range} completed in {time.time() - start:.2f}s\n")

    # ✅ Final sort after last query bin
    write_sorted_results("results_search", "search/")

    print(f"🎉 All SMART searches completed in {time.time() - total_start:.2f}s!")


# --------- USER CONFIG ---------
max_len = max([dicti_inputs[k][0][0, :, :, :].shape[0] for k in dicti_inputs])
print(f"Detected max_len: {max_len}")

main_search_smart(length_bins)


Detected max_len: 1419
Running SMART search | gap: 0.0 | open: 0.0

🆕 Starting new Query Bin: (4, 41)

🟢 Query Bin: (4, 41) x 🔵 Target Bin: (4, 41)
 - Selected 310 queries (range: 4-41) and 310 targets (range: 4-41)
   ➡️  Total query x target pairs: 96100, Batch size: 310
     - Processed 0 pairs...


<ipython-input-55-68740d619ae3>:63: DeprecationWarning: column_stack requires ndarray or scalar arguments, got <class 'list'> at position 0. In a future JAX release this will be an error.
  aln = score_.align_argmax(batch_enc1, batch_enc2, jnp.column_stack((batch_len1, batch_len2)), 1e-7)


 ✅ Bin (4, 41) x (4, 41) completed in 2.08s


🟢 Query Bin: (4, 41) x 🔵 Target Bin: (41, 55)
 - Selected 310 queries (range: 4-41) and 322 targets (range: 41-55)
   ➡️  Total query x target pairs: 99820, Batch size: 310
(55,) (41,)
(41, 55) (41, 55)
     - Processed 0 pairs...
 ✅ Bin (4, 41) x (41, 55) completed in 3.13s


🟢 Query Bin: (4, 41) x 🔵 Target Bin: (55, 63)
 - Selected 310 queries (range: 4-41) and 300 targets (range: 55-63)
   ➡️  Total query x target pairs: 93000, Batch size: 155
(63,) (41,)
(41, 63) (41, 63)
     - Processed 0 pairs...
     - Processed 77500 pairs...
 ✅ Bin (4, 41) x (55, 63) completed in 4.01s


🟢 Query Bin: (4, 41) x 🔵 Target Bin: (63, 70)
 - Selected 310 queries (range: 4-41) and 290 targets (range: 63-70)
   ➡️  Total query x target pairs: 89900, Batch size: 155
     - Processed 0 pairs...
     - Processed 77500 pairs...
 ✅ Bin (4, 41) x (63, 70) completed in 2.86s


🟢 Query Bin: (4, 41) x 🔵 Target Bin: (70, 76)
 - Selected 310 queries (range: 4-41) an